In [1]:
# import necessary libraries
import pandas as pd
import os
import textstat
from openai import OpenAI
import json
import re
import requests
from dotenv import load_dotenv
import math
from IPython.display import FileLink

In [2]:
# import prompts 
from jh_pfx_prompts import example, icd10_example, baseline_zeroshot_prompt, single_fewshot_icd10_labeling_prompt

In [29]:
os.environ['OPENAI_API_KEY'] = ''

In [30]:
# api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLIENT = OpenAI(api_key = OPENAI_API_KEY)
OPENAI_MODEL = "gpt-4o"

In [5]:
#reading levels
PROFESSIONAL = "Professional"
COLLEGE_GRADUATE = "College Graduate"
COLLEGE = "College"
TENTH_TO_TWELTH_GRADE = "10th to 12th grade"
EIGTH_TO_NINTH_GRADE = "8th to 9th grade"
SEVENTH_GRADE = "7th grade"
SIXTH_GRADE = "6th grade"
FIFTH_GRADE = "5th grade"
N_A = "N/A"

In [6]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return PROFESSIONAL
    elif 10.0 <= flesch_reading_ease < 30.0:
        return COLLEGE_GRADUATE
    elif 30.0 <= flesch_reading_ease < 50.0:
        return COLLEGE
    elif 50.0 <= flesch_reading_ease < 60.0:
        return TENTH_TO_TWELTH_GRADE
    elif 60.0 <= flesch_reading_ease < 70.0:
        return EIGTH_TO_NINTH_GRADE
    elif 70.0 <= flesch_reading_ease < 80.0:
        return SEVENTH_GRADE
    elif 80.0 <= flesch_reading_ease < 90.0:
        return SIXTH_GRADE
    elif 90.0 <= flesch_reading_ease < 100.0:
        return FIFTH_GRADE 
    else:
        return N_A

In [7]:
# reading ease variables
fifth_grade = 95
sixth_grade = 85
seventh_grade = 75
eigth_and_ninth_grade = 65
tenth_to_twelfth_grade = 55
college = 40
college_graduate = 20
professional = 5

In [8]:
def adjust_difference(diff, threshold):
    """Adjust the readability difference based on the threshold."""
    if diff > threshold:
        return diff - threshold
    return 0

In [9]:
# import fewshot examples
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [42]:
# import evaluation data 
df_eval = pd.read_csv('pfx_incidental_findings.csv', skiprows = range(1, 313))

In [43]:
df_eval.head()

,Body Part,Organ,Incidental_Finding,ICD-10 Code,ICD-10 Code Description
0,Neck,Lymph Nodes,Lymph Node Calcification,R22.1,"Localized swelling, mass, and lump, neck"
1,Neck,Lymph Nodes,Granulomatous Lymphadenitis,I88.1,Chronic nonspecific lymphadenitis
2,Neck,Lymph Nodes,Lymph Node Enlargement,R59.0,Localized enlarged lymph nodes
3,Neck,Muscles,Fibromatosis Colli,M62.89,Other specified disorders of muscle
4,Neck,Muscles,Muscle Atrophy in Neck,M62.50,"Muscle wasting and atrophy, not elsewhere clas..."


In [11]:
# extract the json from openai
def extract_json(openai_response):
    if openai_response:  # Ensure the response is not None
        try:
            # Extract content from response object
            content = openai_response.message.content
            
            # Search for JSON within the content
            json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)  # Parse JSON string to Python dict
            else:
                print("No JSON found in response content.")
                return None
        except AttributeError as e:
            print(f"Attribute error: {e}. Ensure the input is a valid response object.")
            return None
    else:
        return None

In [12]:
def label_icd10s(pfx_output):
    """
    Takes a single PFx response (string or JSON) and returns
    a labeled ICD-10 result as a Python dictionary (or object).
    """

    # Build up the few-shot examples for ICD-10 labeling
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)

    # Generate the prompt for ICD-10 labeling
    # (Adjust the '{PFx}' if pfx_output is a dictionary with a specific key you need)
    prompt = single_fewshot_icd10_labeling_prompt.format(
        examples=pfx_icd10_fewshot_examples,
        PFx=pfx_output  # or PFx=pfx_output['key'] if needed
    )

    # Call the model to get ICD-10 codes
    pfx_icd10_response = CLIENT.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=0.0,
        messages=[
            {
                "role": "system",
                "content": "You are an ICD10 medical coder for incidental findings. Always respond with a valid JSON object containing the ICD-10 code and its explanation."
            },
            {
                "role": "system",
                "content": prompt
            }
        ],
        stream=False,
    )

    # Extract the JSON structure (or dictionary) from the LLM response
    labeled_result = extract_json(pfx_icd10_response.choices[0])

    return labeled_result


In [44]:
                                                                                                                    # empty list to store results
                                                                                                                    results_df = pd.DataFrame(columns=["finding", "ICD10_code", "PFx", "PFx_ICD10_code"])

# generate each PFx for each row in df_eval
for i, row in df_eval.iterrows():
    prompt = baseline_zeroshot_prompt.format(Incidental_Finding=row['Incidental_Finding'], Reading_Level=SIXTH_GRADE)
    
    pfx_response = CLIENT.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical profesional rephrasing and explaining medical terminology to a patient in an understandable manner."},
            {"role": "system", "content": prompt}
        ],
        stream=False,
    )
    
    # Extract the JSON from the response
    extracted_response = extract_json(pfx_response.choices[0])

    # add information to results
    results_df.loc[i] = {
        "finding": row['Incidental_Finding'],
        "ICD10_code": row['ICD-10 Code'],
        "PFx": extracted_response.get("PFx", ""),
        "PFx_ICD10_code": extracted_response.get("PFx_ICD10_code", "")
    }

In [45]:
len(results_df)

95

In [46]:
# Create a new list to store the labeled ICD10 responses
labeled_icd10_responses = []

# Iterate over each response in pfx_zeroshot_output_all_df and apply the label_icd10s function
for response in results_df['PFx']:
    labeled_icd10_responses.append(label_icd10s(response))

In [47]:
# Create lists to store the results
agent_icd10_codes = []
icd10_matches = []
pfx_icd10_matches = []
flesch_scores = []

agent_icd10_codes.extend([list(x.values())[0] if x else "" for x in labeled_icd10_responses])

for index, row in results_df.iterrows():
    # Compare to the "ICD10_code" in your DataFrame (if it exists)
    agent_icd10_code = agent_icd10_codes[index]
    icd10_match = (row["ICD10_code"] == agent_icd10_code)
    icd10_matches.append(icd10_match)

    # compare 
    pfx_icd10_match = (row["PFx_ICD10_code"] == row["ICD10_code"])
    pfx_icd10_matches.append(pfx_icd10_match)

    # Calculate the Flesch Reading Ease score
    flesch_score = textstat.flesch_reading_ease(row['PFx'])
    flesch_scores.append(flesch_score)

# Add the results to the DataFrame
results_df['_0_agent_icd10_codes'] = agent_icd10_codes
results_df['_0_icd10_matches'] = icd10_matches
results_df['_0_pfx_icd10_matches'] = pfx_icd10_matches
results_df['_0_flesch'] = flesch_scores

In [48]:
desired_reading_ease = sixth_grade
# Calculate threshold for penalty
if desired_reading_ease >= 55:
    threshold = 10
else:
    threshold = 20

# Create lists to store the results
accuracy_icd10_matches_list = []
accuracy_pfx_matches_list = []
readability_difference_list = []
overall_score_list = []
log_overall_score_list = []

# Iterate over each row in the DataFrame
for index, row in results_df.iterrows():
    # Calculate accuracy scores
    accuracy_icd10_matches = row["_0_icd10_matches"]
    accuracy_pfx_matches = row["_0_pfx_icd10_matches"]
    flesch_score = row["_0_flesch"]

    # total number of icd10 matches
    total_icd10_matches = accuracy_icd10_matches + accuracy_pfx_matches

    # Adjust weights for overall score
    # Calculate readability score 
    readability_score = flesch_score
    readability_difference = abs(readability_score - desired_reading_ease)

    # Compute the overall score
    overall_score = total_icd10_matches * 0.8  + 0.2 * (1/(readability_difference + 1))

    # Calculate readability score
    readability_difference_log = desired_reading_ease - flesch_score
    if readability_difference_log <= threshold:  # No penalty if difference is within the threshold
        readability_difference_p = 0
    else:  # Apply penalty only if readability exceeds the threshold
        readability_difference_with_threshold = readability_difference_log - threshold
        readability_difference_p = math.log(1 + readability_difference_with_threshold) / math.log(20)

    log_overall_score = total_icd10_matches * 0.8 + readability_difference_log * 0.2

    # Append results to lists
    accuracy_icd10_matches_list.append(float(accuracy_icd10_matches))
    accuracy_pfx_matches_list.append(float(accuracy_pfx_matches))
    readability_difference_list.append(float(readability_difference))
    overall_score_list.append(float(overall_score))
    log_overall_score_list.append(float(log_overall_score))
    

# Create a DataFrame with the results
grades_data = {
    "accuracy_agent_icd10": accuracy_icd10_matches_list,
    "accuracy_pfx_icd10": accuracy_pfx_matches_list,
    "readability_difference": readability_difference_list,
    "overall_score": overall_score_list,
    "log_overall_score": log_overall_score_list,
}
grades = pd.DataFrame(grades_data)
results_df = pd.concat([results_df, grades], axis=1)


In [49]:
results_df.head()

,finding,ICD10_code,PFx,PFx_ICD10_code,_0_agent_icd10_codes,_0_icd10_matches,_0_pfx_icd10_matches,_0_flesch,accuracy_agent_icd10,accuracy_pfx_icd10,readability_difference,overall_score,log_overall_score
0,Lymph Node Calcification,R22.1,Lymph node calcification means that some of yo...,Z86.19,R91.8,False,False,79.80,0.0,0.0,5.20,0.032258,1.040
1,Granulomatous Lymphadenitis,I88.1,Granulomatous lymphadenitis is a condition whe...,R59.0,D86.89,False,False,60.85,0.0,0.0,24.15,0.007952,4.830
2,Lymph Node Enlargement,R59.0,"Lymph nodes are small, bean-shaped structures ...",R59.9,R59.0,True,False,83.56,1.0,0.0,1.44,0.881967,1.088
3,Fibromatosis Colli,M62.89,Fibromatosis colli is a condition that affects...,M62.89,M62.89,True,True,75.61,1.0,1.0,9.39,1.619249,3.478
4,Muscle Atrophy in Neck,M62.50,Muscle atrophy in the neck means that the musc...,R29.890,M62.50,True,False,69.52,1.0,0.0,15.48,0.812136,3.896


In [52]:
results_df.to_csv('PFx_zeroshot_95.csv', index = False)

In [53]:
FileLink('PFx_zeroshot_95.csv')

C:\Users\User\VS Code Projects\Science Research\jh_pfx\jh_main\PFx_zeroshot_95.csv